# Exploring the Spotify API

We can use the Spotify API to obtain interesting music-related data. We could potentially combine that with other data sources to get a nice dataset to work with for this project.

To access the API we need to have a Spotify developer account and register a new application to obtain a client ID and secret. For this notebook, we have already done that and added a `.env` file with the following content:
```
SPOTIPY_CLIENT_ID='our-client-id-would-be-here'
SPOTIPY_CLIENT_SECRET='our-client-secret-would-be-here'
SPOTIPY_REDIRECT_URI='http://127.0.0.1:9090'
```

These environment variables will be used by the `spotipy` package, a lightweight python library for getting data from the Spotify API.

We also added the redirect URL `http://127.0.0.1:9090` mentioned in the `SPOTIPY_REDIRECT_URI` variable to the application in the Developer Console on the Spotify website. spotipy will "instantiate a server on the indicated response to receive the access token from the response at the end of the oauth flow" (as mentioned in the [docs](https://spotipy.readthedocs.io/en/2.21.0/#redirect-uri))

To load the environment variables from that file we use `load_dotenv`:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Now, we are ready to import spotipy:

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

Let's load the duplicated tracks we identified `merge_crawled_and_existing_chart_data.ipynb`

In [3]:
import pandas as pd
from helpers import get_data_path, split_dataframe

duplicates = pd.read_csv(get_data_path("duplicates.csv"))
duplicates

,region,date,rank,title_existing,artist_existing,streams_existing,uri_existing,title_new,artist_new,streams_new,uri_new
0,Slovakia,2017-01-05,16,All Night,"The Vamps, Matoma",1326.0,spotify:track:1yNyoWWWikbLhwIGWjZuDW,All Night,"The Vamps, Matoma",1326.0,spotify:track:4b4KcovePX8Ke2cLIQTLM0
1,Slovakia,2017-01-05,23,Now and Later,Sage The Gemini,1224.0,spotify:track:43jBqV3j3Xi1g6wO0bhIMd,Now and Later,Sage The Gemini,1224.0,spotify:track:08WPvDEsHvTFuB9w8tC2OS
2,Slovakia,2017-01-06,19,All Night,"The Vamps, Matoma",1278.0,spotify:track:1yNyoWWWikbLhwIGWjZuDW,All Night,"The Vamps, Matoma",1278.0,spotify:track:4b4KcovePX8Ke2cLIQTLM0
3,Slovakia,2017-01-06,23,Now and Later,Sage The Gemini,1215.0,spotify:track:43jBqV3j3Xi1g6wO0bhIMd,Now and Later,Sage The Gemini,1215.0,spotify:track:08WPvDEsHvTFuB9w8tC2OS
4,Slovakia,2017-01-07,22,Now and Later,Sage The Gemini,1291.0,spotify:track:43jBqV3j3Xi1g6wO0bhIMd,Now and Later,Sage The Gemini,1291.0,spotify:track:08WPvDEsHvTFuB9w8tC2OS
...,...,...,...,...,...,...,...,...,...,...,...
149,Slovakia,2017-05-10,35,You Don't Know Me - Radio Edit,"Jax Jones, RAYE",1409.0,spotify:track:4WrS6vezaPwOafKJmHqMEU,You Don't Know Me (feat. RAYE),"Jax Jones, RAYE",1409.0,spotify:track:00lNx0OcTJrS3MKHcB80HY
150,Slovakia,2017-05-10,37,Passionfruit,Drake,1398.0,spotify:track:5mCPDVBb16L4XQwDdbRUpz,Passionfruit,Drake,1398.0,spotify:track:7hDc8b7IXETo14hHIHdnhd
151,Slovakia,2017-05-10,43,DNA.,Kendrick Lamar,1112.0,spotify:track:6HZILIRieu8S0iqY8kIKhj,SUBEME LA RADIO (feat. Descemer Bueno & Zion &...,"Enrique Iglesias, Descemer Bueno, Zion & Lennox",1105.0,spotify:track:7nKBxz47S9SD79N086fuhn
152,Slovakia,2017-05-10,46,Bon Appétit,"Katy Perry, Migos",1036.0,spotify:track:14oOKFGuXu8DqvOe1W8vao,Bon Appétit,"Katy Perry, Migos",1036.0,spotify:track:61WbtB6ujkpNAsAf5LjF4b


In [8]:
chunks = split_dataframe(duplicates, chunk_size=50)

In [13]:
existing_tracks = pd.concat([pd.DataFrame(sp.tracks(chunk.uri_existing)["tracks"]) for chunk in chunks])
existing_tracks

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1,197793,False,{'isrc': 'GBUM71605342'},{'spotify': 'https://open.spotify.com/track/1y...,https://api.spotify.com/v1/tracks/1yNyoWWWikbL...,1yNyoWWWikbLhwIGWjZuDW,False,All Night,0,None,1,track,spotify:track:1yNyoWWWikbLhwIGWjZuDW
1,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,193181,False,{'isrc': 'USAT21602883'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/43jBqV3j3Xi1...,43jBqV3j3Xi1g6wO0bhIMd,False,Now and Later,54,https://p.scdn.co/mp3-preview/4e07add7f65df6d1...,1,track,spotify:track:43jBqV3j3Xi1g6wO0bhIMd
2,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1,197793,False,{'isrc': 'GBUM71605342'},{'spotify': 'https://open.spotify.com/track/1y...,https://api.spotify.com/v1/tracks/1yNyoWWWikbL...,1yNyoWWWikbLhwIGWjZuDW,False,All Night,0,None,1,track,spotify:track:1yNyoWWWikbLhwIGWjZuDW
3,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,193181,False,{'isrc': 'USAT21602883'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/43jBqV3j3Xi1...,43jBqV3j3Xi1g6wO0bhIMd,False,Now and Later,54,https://p.scdn.co/mp3-preview/4e07add7f65df6d1...,1,track,spotify:track:43jBqV3j3Xi1g6wO0bhIMd
4,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,193181,False,{'isrc': 'USAT21602883'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/43jBqV3j3Xi1...,43jBqV3j3Xi1g6wO0bhIMd,False,Now and Later,54,https://p.scdn.co/mp3-preview/4e07add7f65df6d1...,1,track,spotify:track:43jBqV3j3Xi1g6wO0bhIMd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1,211935,True,{'isrc': 'GBUM71606456'},{'spotify': 'https://open.spotify.com/track/4W...,https://api.spotify.com/v1/tracks/4WrS6vezaPwO...,4WrS6vezaPwOafKJmHqMEU,False,You Don't Know Me - Radio Edit,0,None,1,track,spotify:track:4WrS6vezaPwOafKJmHqMEU
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,298940,True,{'isrc': 'USCM51700072'},{'spotify': 'https://open.spotify.com/track/5m...,https://api.spotify.com/v1/tracks/5mCPDVBb16L4...,5mCPDVBb16L4XQwDdbRUpz,False,Passionfruit,82,https://p.scdn.co/mp3-preview/c014129d2fde9be4...,3,track,spotify:track:5mCPDVBb16L4XQwDdbRUpz
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,185946,True,{'isrc': 'USUM71703079'},{'spotify': 'https://open.spotify.com/track/6H...,https://api.spotify.com/v1/tracks/6HZILIRieu8S...,6HZILIRieu8S0iqY8kIKhj,False,DNA.,79,https://p.scdn.co/mp3-preview/7c9aac4ace7e49d4...,2,track,spotify:track:6HZILIRieu8S0iqY8kIKhj
2,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1,227292,False,{'isrc': 'USUM71702488'},{'spotify': 'https://open.spotify.com/track/14...,https://api.spotify.com/v1/tracks/14oOKFGuXu8D...,14oOKFGuXu8DqvOe1W8vao,False,Bon Appétit,0,None,1,track,spotify:track:14oOKFGuXu8DqvOe1W8vao


In [14]:
new_tracks = pd.concat([pd.DataFrame(sp.tracks(chunk.uri_new)["tracks"]) for chunk in chunks])
new_tracks

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1,197640,False,{'isrc': 'GBUM71605342'},{'spotify': 'https://open.spotify.com/track/4b...,https://api.spotify.com/v1/tracks/4b4KcovePX8K...,4b4KcovePX8Ke2cLIQTLM0,False,All Night,0,None,2,track,spotify:track:4b4KcovePX8Ke2cLIQTLM0
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,193181,False,{'isrc': 'USAT21702430'},{'spotify': 'https://open.spotify.com/track/08...,https://api.spotify.com/v1/tracks/08WPvDEsHvTF...,08WPvDEsHvTFuB9w8tC2OS,False,Now and Later,54,https://p.scdn.co/mp3-preview/4e07add7f65df6d1...,15,track,spotify:track:08WPvDEsHvTFuB9w8tC2OS
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1,197640,False,{'isrc': 'GBUM71605342'},{'spotify': 'https://open.spotify.com/track/4b...,https://api.spotify.com/v1/tracks/4b4KcovePX8K...,4b4KcovePX8Ke2cLIQTLM0,False,All Night,0,None,2,track,spotify:track:4b4KcovePX8Ke2cLIQTLM0
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,193181,False,{'isrc': 'USAT21702430'},{'spotify': 'https://open.spotify.com/track/08...,https://api.spotify.com/v1/tracks/08WPvDEsHvTF...,08WPvDEsHvTFuB9w8tC2OS,False,Now and Later,54,https://p.scdn.co/mp3-preview/4e07add7f65df6d1...,15,track,spotify:track:08WPvDEsHvTFuB9w8tC2OS
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,193181,False,{'isrc': 'USAT21702430'},{'spotify': 'https://open.spotify.com/track/08...,https://api.spotify.com/v1/tracks/08WPvDEsHvTF...,08WPvDEsHvTFuB9w8tC2OS,False,Now and Later,54,https://p.scdn.co/mp3-preview/4e07add7f65df6d1...,15,track,spotify:track:08WPvDEsHvTFuB9w8tC2OS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1,213946,True,{'isrc': 'GBUM71606456'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/00lNx0OcTJrS...,00lNx0OcTJrS3MKHcB80HY,False,You Don't Know Me - Radio Edit,0,None,1,track,spotify:track:00lNx0OcTJrS3MKHcB80HY
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1,298940,True,{'isrc': 'USCM51700072'},{'spotify': 'https://open.spotify.com/track/7h...,https://api.spotify.com/v1/tracks/7hDc8b7IXETo...,7hDc8b7IXETo14hHIHdnhd,False,Passionfruit,0,None,3,track,spotify:track:7hDc8b7IXETo14hHIHdnhd
1,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,207680,False,{'isrc': 'USRC11700058'},{'spotify': 'https://open.spotify.com/track/7n...,https://api.spotify.com/v1/tracks/7nKBxz47S9SD...,7nKBxz47S9SD79N086fuhn,False,SUBEME LA RADIO (feat. Descemer Bueno & Zion &...,69,https://p.scdn.co/mp3-preview/1b7a210a0e85e975...,1,track,spotify:track:7nKBxz47S9SD79N086fuhn
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1,227821,False,{'isrc': 'USUM71702488'},{'spotify': 'https://open.spotify.com/track/61...,https://api.spotify.com/v1/tracks/61WbtB6ujkpN...,61WbtB6ujkpNAsAf5LjF4b,False,Bon appétit,0,None,11,track,spotify:track:61WbtB6ujkpNAsAf5LjF4b


In [ ]:
type(example)

dict

In [ ]:
[key for key in example.keys()]

['album',
 'artists',
 'available_markets',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [ ]:
example["artists"]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0EmeFodog0BfCgMzAIvKQp'},
  'href': 'https://api.spotify.com/v1/artists/0EmeFodog0BfCgMzAIvKQp',
  'id': '0EmeFodog0BfCgMzAIvKQp',
  'name': 'Shakira',
  'type': 'artist',
  'uri': 'spotify:artist:0EmeFodog0BfCgMzAIvKQp'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/1r4hJ1h58CWwUQe3MxPuau'},
  'href': 'https://api.spotify.com/v1/artists/1r4hJ1h58CWwUQe3MxPuau',
  'id': '1r4hJ1h58CWwUQe3MxPuau',
  'name': 'Maluma',
  'type': 'artist',
  'uri': 'spotify:artist:1r4hJ1h58CWwUQe3MxPuau'}]